## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [2]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [3]:
# View the column names in the data
df.columns

Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [4]:
# Use the describe function to gather some basic statistics
df.describe

<bound method NDFrame.describe of            first       last                      job               phone  \
0         Donald    Harding             Immunologist  793-904-7725x39308   
1        Tiffany      Myers          Music therapist    201.442.4543x942   
2        Shannon     Watson             Immunologist   687.737.9424x8503   
3         Nathan      Baker    Accounting technician    827-788-8123x012   
4      Christina   Schwartz             Chiropractor        265-829-3643   
...          ...        ...                      ...                 ...   
54634     Lauren      Reese           Radio producer  977.877.5272x11382   
54635    Derrick      Moore                 Musician        358.661.5483   
54636     Monica  Gutierrez         Graphic designer    294.805.9100x339   
54637      Wanda    Solomon             Toxicologist       (311)767-4924   
54638     Dwayne       Bell  Chief Financial Officer   606-619-7667x9508   

                                     email  client_id

In [ ]:
# Use this space to do any additional research
# and familiarize yourself with the data.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54639 entries, 0 to 54638
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   first        54639 non-null  object 
 1   last         54639 non-null  object 
 2   job          54639 non-null  object 
 3   phone        54639 non-null  object 
 4   email        54639 non-null  object 
 5   client_id    54639 non-null  int64  
 6   order_id     54639 non-null  int64  
 7   order_date   54639 non-null  object 
 8   order_week   54639 non-null  int64  
 9   order_year   54639 non-null  int64  
 10  item_id      54639 non-null  object 
 11  category     54639 non-null  object 
 12  subcategory  54639 non-null  object 
 13  unit_price   54639 non-null  float64
 14  unit_cost    54639 non-null  float64
 15  unit_weight  54639 non-null  float64
 16  qty          54639 non-null  int64  
 17  line_number  54639 non-null  int64  
dtypes: float64(3), int64(6), object(9)
memory usag

In [7]:
# What three item categories had the most entries?
df['category'].value_counts().head(3)

category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [9]:
# For the category with the most entries, which subcategory had the most entries?
df.groupby('category')['subcategory'].value_counts().loc[df['category'].value_counts().idxmax()].head(1)


subcategory
bathroom supplies    6424
Name: count, dtype: int64

In [10]:
# Which five clients had the most entries in the data?
df['client_id'].value_counts().head(5)

client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [14]:
# Store the client ids of those top 5 clients in a list.
top_clients_list = list(df['client_id'].value_counts().index[:5])

print(top_clients_list)


[33615, 66037, 46820, 38378, 24741]


In [20]:
# How many total units (the qty column) did the client with the most entries order order?
df.groupby('client_id')['qty'].sum().loc[df['client_id'].value_counts().idxmax()]



64313

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [23]:
# Create a column that calculates the subtotal for each line using the unit_price and the qty
df['line_subtotal'] = df['unit_price'] * df['qty']

print(df[['unit_price', 'qty', 'line_subtotal']].head(2))

   unit_price  qty  line_subtotal
0     1096.80  105      115164.00
1       24.95   21         523.95


In [27]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.
df['total_weight'] = df['unit_weight'] * df['qty']

df['shipping_price'] = df['total_weight'].apply(lambda x: x * 7 if x > 50 else x * 10)

print(df[['unit_price', 'unit_weight', 'qty', 'total_weight', 'shipping_price']]. head())

   unit_price  unit_weight  qty  total_weight  shipping_price
0     1096.80         7.50  105        787.50         5512.50
1       24.95         1.49   21         31.29          312.90
2       13.52         1.68   39         65.52          458.64
3       36.42         1.23   29         35.67          356.70
4      195.10        46.43   20        928.60         6500.20


In [28]:
# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%
df['line_price'] = (df['line_subtotal'] + df['shipping_price']) * 1.0925

print(df[['line_subtotal', 'shipping_price', 'line_price']].head(3))

   line_subtotal  shipping_price     line_price
0      115164.00         5512.50  131839.076250
1         523.95          312.90     914.258625
2         527.28          458.64    1077.117600


In [ ]:
# Create a column for the cost of each line using unit cost, qty, and
# shipping price (assume the shipping cost is exactly what is charged to the client).
df['line_cost'] = (df['unit_cost'] * df['qty']) + df['shipping_price']

df.head(3)

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,line_price,line_cost
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.076250,85597.05
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,31.29,312.90,914.258625,629.79
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,65.52,458.64,1077.117600,765.18


In [30]:
# Create a column for the profit of each line using line cost and line price
df['line_profit'] = df['line_price'] - df['line_cost']

df.head(3)


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,line_price,line_cost,line_profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.076250,85597.05,46242.026250
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,15.09,1.49,21,0,523.95,31.29,312.90,914.258625,629.79,284.468625
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,7.86,1.68,39,6,527.28,65.52,458.64,1077.117600,765.18,311.937600


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [31]:
# Check your work using the totals above
order_totals = df.groupby('order_id')['line_price'].sum()

# Verify the totals for the given order IDs
order_2742071 = order_totals.loc[2742071]
order_2173913 = order_totals.loc[2173913]
order_6128929 = order_totals.loc[6128929]

# Print the results
print(f"Order 2742071 total: ${order_2742071:.2f}")
print(f"Order 2173913 total: ${order_2173913:.2f}")
print(f"Order 6128929 total: ${order_6128929:.2f}")


Order 2742071 total: $152811.90
Order 2173913 total: $162388.72
Order 6128929 total: $923441.24


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [32]:
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.
top_client_spending = {client_id: df[df['client_id'] == client_id]['line_price'].sum() for client_id in top_clients_list}

# Print the results
for client_id, total_spent in top_client_spending.items():
    print(f"{client_id}: ${total_spent:.2f}")



33615: $8377308.57
66037: $10259514.80
46820: $9743794.32
38378: $12906550.88
24741: $82268891.98


In [ ]:
# Create a summary DataFrame showing the totals for the for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit. 
summary_df = df[df['client_id'].isin(top_clients_list)].groupby('client_id')[['qty', 'shipping_price', 'line_price', 'line_cost', 'line_profit']].sum().reset_index()

# Display the summary DataFrame
print(summary_df)

   client_id     qty  shipping_price    line_price    line_cost   line_profit
0      24741  239862      5126448.37  8.226889e+07  45688899.71  3.657999e+07
1      33615   64313      1828984.89  8.377309e+06   6175313.91  2.201995e+06
2      38378   73667      3429455.40  1.290655e+07   9634720.98  3.271830e+06
3      46820   75768      1601448.84  9.743794e+06   7007191.64  2.736603e+06
4      66037   43018      1395151.85  1.025951e+07   7004482.98  3.255032e+06


In [35]:
# Format the data and rename the columns to names suitable for presentation.

# Define the money columns
money_columns = ['shipping_price', 'line_price', 'line_cost', 'line_profit']

# Define a function to convert a dollar amount to millions
def currency_format_millions(value):
    return value / 1_000_000

# Apply the currency_format_millions function to the money columns
for col in money_columns:
    summary_df[col] = summary_df[col].apply(currency_format_millions)

# Rename the columns to names suitable for presentation
summary_df.rename(columns={
    'client_id': 'Client ID',
    'qty': 'Units',
    'shipping_price': 'Shipping (millions)',
    'line_price': 'Total Revenue (millions)',
    'line_cost': 'Total Cost (millions)',
    'line_profit': 'Total Profit (millions)'
}, inplace=True)

# Display the formatted DataFrame
print(summary_df)





   Client ID   Units  Shipping (millions)  Total Revenue (millions)  \
0      24741  239862             5.126448                 82.268892   
1      33615   64313             1.828985                  8.377309   
2      38378   73667             3.429455                 12.906551   
3      46820   75768             1.601449                  9.743794   
4      66037   43018             1.395152                 10.259515   

   Total Cost (millions)  Total Profit (millions)  
0              45.688900                36.579992  
1               6.175314                 2.201995  
2               9.634721                 3.271830  
3               7.007192                 2.736603  
4               7.004483                 3.255032  


In [36]:
# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.
# Sort the data by "Total Profit (millions)" in descending order
sorted_summary_df = summary_df.sort_values(by='Total Profit (millions)', ascending=False)

# Display the sorted DataFrame
print(sorted_summary_df)


   Client ID   Units  Shipping (millions)  Total Revenue (millions)  \
0      24741  239862             5.126448                 82.268892   
2      38378   73667             3.429455                 12.906551   
4      66037   43018             1.395152                 10.259515   
3      46820   75768             1.601449                  9.743794   
1      33615   64313             1.828985                  8.377309   

   Total Cost (millions)  Total Profit (millions)  
0              45.688900                36.579992  
2               9.634721                 3.271830  
4               7.004483                 3.255032  
3               7.007192                 2.736603  
1               6.175314                 2.201995  
